### Test of data instantiation

In [ ]:
import torch
import numpy
from torch.utils.data import Dataset, DataLoader 
from torchvision.datasets import ImageFolder
import pandas as pd
from PIL import Image


In [4]:

class ExpDataSet(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = torch.as_tensor(images)
        self.labels = torch.as_tensor(labels)
        self.transform = transform
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]

        if self.transform: # Add transformation mask to image
            image = self.transform(image)
        
        return image, label
        
img = torch.ones(6, 3)
labels = [0, 1, 1, 1, 0, 0]

dataset = ExpDataSet(img, labels)
print(dataset[4])



(tensor([1., 1., 1.]), tensor(0))


In [5]:
# Local paths

label_path = "dataset/datasets/ringmerkingno/ringcodes.csv"





# Make sure to separate csv file on |
df = pd.read_csv(label_path, sep="|")

df.head()

,filename,code,color
0,LBNW(J22M)_20080310234907_79.jpg,J22M,NW
1,LBNW(J03Y)_20080311094828_79.jpg,J03Y,NW
2,LBBW(JR0T)_20080320193604_79.jpg,JR0T,BW
3,LBBW(J1EM)_20080321135128_79.jpg,J1EM,BW
4,LBBW(JZ3K)_20111027125715_79.jpg,JZ3K,BW


In [9]:

dl = df.sort_values("filename", ascending=True).reset_index(drop=True)
dl.head()


,filename,code,color
0,LABW(WT);LBM_20120228142549_79.jpg,WT,BW
1,LAM;RBWN(J0K8)_20140125233654_1116.jpg,J0K8,WN
2,LAM;RBWN(J0K8)_20190120000322_1116.jpg,J0K8,WN
3,LAM;RBWN(JA0H)_20121029120844_1116.jpg,JA0H,WN
4,LANW(CN);LBM_20120227142458_79.jpg,CN,NW


In [7]:
image_path = "dataset/datasets/ringmerkingno/images"
exp_x = ...



In [8]:
exp_loader = ExpDataSet()

TypeError: ExpDataSet.__init__() missing 2 required positional arguments: 'images' and 'labels'